In [136]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F

In [127]:
CLASSES=10

class NiN(nn.Module):
    def __init__(self):
        super(NiN,self).__init__()
        self.classifier=nn.Sequential(
            nn.Conv2d(1,96,kernel_size=11,stride=4,padding=0),
            nn.ReLU(inplace=True),
            nn.Conv2d(96,96,kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(96,96,kernel_size=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3,stride=2),
            nn.Dropout(0.5),
        
            nn.Conv2d(96,256,kernel_size=5,stride=1,padding=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(256,256,kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256,256,kernel_size=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3,stride=2),
            nn.Dropout(0.5),
        
            nn.Conv2d(256,384,kernel_size=3,stride=1,padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384,384,kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384,384,kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            
            nn.Conv2d(384,10,kernel_size=3,stride=1,padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(10,10,kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(10,10,kernel_size=1),
            nn.ReLU(inplace=True),
                       
            nn.AdaptiveAvgPool2d((1, 1))
        )
    
    def forward(self,x):
        x=self.classifier(x)
        x=x.view(x.size(0),CLASSES)      
        return x

In [74]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.classifier = nn.Sequential(
                nn.Conv2d(1, 192, kernel_size=5, stride=1, padding=2),
                nn.ReLU(inplace=True),
                nn.Conv2d(192, 160, kernel_size=1, stride=1, padding=0),
                nn.ReLU(inplace=True),
                nn.Conv2d(160,  96, kernel_size=1, stride=1, padding=0),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
                nn.Dropout(0.5),

                nn.Conv2d(96, 192, kernel_size=5, stride=1, padding=2),
                nn.ReLU(inplace=True),
                nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0),
                nn.ReLU(inplace=True),
                nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0),
                nn.ReLU(inplace=True),
                nn.AvgPool2d(kernel_size=3, stride=2, padding=1),
                nn.Dropout(0.5),

                nn.Conv2d(192, 192, kernel_size=3, stride=1, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0),
                nn.ReLU(inplace=True),
                nn.Conv2d(192,  10, kernel_size=1, stride=1, padding=0),
                nn.ReLU(inplace=True),
                nn.AvgPool2d(kernel_size=8, stride=1, padding=0),

                )

    def forward(self, x):
        x = self.classifier(x)
        x = x.view(x.size(0), 10)
        return x

In [110]:
CLASSES=10
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [111]:
transform_conf=transforms.Compose([transforms.Resize((224,224)),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.1307,),(0.3081,))])

In [11]:
#ransform_conf=transforms.Compose([transforms.ToTensor()])
                                 

In [112]:
BATCH_SIZE=64
train_dataset=datasets.FashionMNIST('/home/ubuntu/gpu_work/',train=True,download=True,transform=transform_conf)
test_dataset=datasets.FashionMNIST('/home/ubuntu/gpu_work/',train=True,download=True,transform=transform_conf)

In [113]:
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=BATCH_SIZE,shuffle=True)

In [128]:
model=NiN().to(device)
#model=Net().to(device)
optimizer=optim.Adam(params=model.parameters(),lr=0.0001)
loss_fn=nn.CrossEntropyLoss()

In [133]:
def train(model,train_loader,optimizer,device,epoch):
    model.train()
    for batch_ids, (data,classes) in enumerate(train_loader):
        classes=classes.type(torch.LongTensor)
        data,classes=data.to(device),classes.to(device)
        
        optimizer.zero_grad()
        output=model(data)
        loss=loss_fn(output,classes)
        
        loss.backward()
        optimizer.step()
        
        if(batch_ids+1) % 500 == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_ids* len(data), len(train_loader.dataset),
                100.*batch_ids / len(train_loader),loss.item()))

In [134]:
def test(model,test_loader,device):
    model.eval()
    test_loss=0
    correct=0
    with torch.no_grad():
        for data, classes in test_loader:
            data, classes = data.to(device), classes.to(device)
            y_hat=model(data)
            _,y_pred=torch.max(y_hat,1)
            correct+=(y_pred==classes).sum().item()
            test_loss+=F.nll_loss(y_hat,classes,reduction='sum').item()
        test_loss/=len(test_dataset)
        print("\n Test set: Avarage loss: {:.0f},Accuracy:{}/{} ({:.0f}%)\n".format(
            test_loss,correct,len(test_dataset),100.*correct/len(test_dataset)))
        print('='*30)

In [137]:
if __name__=='__main__':
    seed=42
    EPOCHS=10
    
    for epoch in range(1,EPOCHS+1):
        train(model,train_loader,optimizer,device,epoch)
        test(model,test_loader,device)

Train Epoch: 1 [31936/60000 (53%)]	Loss: 0.685712

 Test set: Avarage loss: -17,Accuracy:47438/60000 (79%)

Train Epoch: 2 [31936/60000 (53%)]	Loss: 0.547450

 Test set: Avarage loss: -17,Accuracy:47948/60000 (80%)

Train Epoch: 3 [31936/60000 (53%)]	Loss: 0.433725

 Test set: Avarage loss: -21,Accuracy:48892/60000 (81%)

Train Epoch: 4 [31936/60000 (53%)]	Loss: 0.522983

 Test set: Avarage loss: -21,Accuracy:49819/60000 (83%)

Train Epoch: 5 [31936/60000 (53%)]	Loss: 0.724332

 Test set: Avarage loss: -20,Accuracy:50247/60000 (84%)

Train Epoch: 6 [31936/60000 (53%)]	Loss: 0.310877

 Test set: Avarage loss: -20,Accuracy:50280/60000 (84%)

Train Epoch: 7 [31936/60000 (53%)]	Loss: 0.608101

 Test set: Avarage loss: -21,Accuracy:50780/60000 (85%)

Train Epoch: 8 [31936/60000 (53%)]	Loss: 0.396246

 Test set: Avarage loss: -23,Accuracy:51275/60000 (85%)

Train Epoch: 9 [31936/60000 (53%)]	Loss: 0.396898

 Test set: Avarage loss: -21,Accuracy:51435/60000 (86%)

Train Epoch: 10 [31936/60000

In [19]:
mat=torch.rand(64,10,50,50)

In [139]:
import gc
#del model
gc.collect()
torch.cuda.empty_cache()

In [24]:
print(mat.size(0))
temp=mat.view(64,-1)

64


In [20]:
fn=nn.Flatten()
temp=fn(mat)


In [21]:
temp.shape

torch.Size([64, 25000])

In [58]:
from torchsummary import summary



ModuleNotFoundError: No module named 'torchsummary'

In [107]:
# pool of square window of size=3, stride=2
#m = nn.AdaAvgPool2d(7, stride=3)
# pool of non-square window
m = nn.AdaptiveAvgPool2d((1, 1))
input = torch.randn(1, 16, 80, 80)
output = m(input)

In [108]:
output.shape

torch.Size([1, 16, 1, 1])